# LLMs - Using the OpenAI API
by Tobias Heimig-Elschner (BBSR) - 2025

In this notebook, we will use the OpenAI Python client library to interact with the OpenAI API.
We will establish a connection to the API and use it to:
- Get embeddings for a given text
- Perform single-turn completions
- Create multi-turn conversations

### Pre-requisites
- OpenAI API key (the API key should be provided as an environment variable OPENAI_API_KEY) 
- **NOTE**: API keys are sensitive credentials and should never be hard-coded or shared. They should be stored securely and accessed programmatically.
- Python client library for the OpenAI API `openai`

### Further Reading
- [OpenAI Cookbook](https://cookbook.openai.com/)
- [OpenAI API Documentation](https://platform.openai.com/docs/api-reference/introduction)
- [OpenAI Python Client Library README](https://github.com/openai/openai-python/blob/main/README.md)
- [OpenAI Python Client Library API Reference](https://github.com/openai/openai-python/blob/main/api.md)
- [OpenAI Python Client Library Examples](https://github.com/openai/openai-python/tree/main/examples)

---

## Setup

This section shows how to set up the OpenAI client for interacting with the OpenAI API.

### Task
- Use the OpenAI Python library (`openai`) to create an OpenAI client instance with `openai.Client`.

### Tips
- Use the `openai.Client` class to create a client object.
- The API key can be accessed using the environment variable `OPENAI_API_KEY` with  `os.getenv('OPENAI_API_KEY')`
- Providing the API key is optional, because the client uses the `OPENAI_API_KEY` environment variable by default if no key is explicitly passed.

In [ ]:
import openai
import os

# OPTIONAL: Load OpenAI API key manually (default behavior already reads from env)
api_key = os.getenv("OPENAI_API_KEY")

# Get an openai-client
ai_client = openai.Client() # openai.Client(api_key) if you want to set it explicitly

## Utilizing Embedding Models - Vectorization of Data

In this section, we will use the OpenAI API to get embeddings for a given text - get the embedding of 'OECD'.

### Task
- Use the OpenAI API Endpoint `embeddings` to get the embeddings for 'OECD'.

### Tips
- Use the `Client.embeddings.create` method.
- Use the model: 'text-embedding-3-small' as the embedding model.
- Specify the text you want to embed as the `input` variable in the `Client.embeddings.create` method.
- Specify the model using the `model` variable in the `Client.embeddings.create` method.
- The embeddings are returned as a list of floats and is returned in a `CreateEmbeddingResponse` object.
    - The embeddings can be accessed using the `CreateEmbeddingResponse.data` attribute which contains the list of objects.
    - The first element of the list is the actual embedding object.
    - The embedding (a list of floats) can be accessed using the `CreateEmbeddingResponse.data[0].embedding` attribute.

In [ ]:
# Specify model and text
model = "text-embedding-3-small"
text = "OECD"

# Query the embedding
response = ai_client.embeddings.create(
    model = model,
    input = text
)

#Getting the embedding
embedding = response.data[0].embedding
print(f"Embedding: {embedding[:10]}")

### Further information - Inspecting `CreateEmbeddingResponse`
- `CreateEmbeddingResponse.data` &rarr; a list of embeddings (`Embedding`); list because the API endpoints allows to create multiple embeddings in one request as long as the number of max tokens is not exceeded.
- `CreateEmbeddingResponse.model` &rarr; the name (`str`) of the model used
- `CreateEmbeddingResponse.object` &rarr; the type (`str`) of the data-object that is returned for the request (for embeddings-endpoint always 'list')
- `CreateEmbeddingResponse.usage` &rarr; a usage-obj(`Usage`) containing information about the used tokens (prompt_tokens and total_tokens)

## Utilizing Chat Models - Single Turn

In this section, we will use the OpenAI API to get a single-turn completion for a given user prompt.

### Task
- Use the OpenAI API endpoint `completions` to get a single-turn completion for a given user prompt – "What does 'OECD' stand for?"

### Tips
- Use the `Client.chat.completions.create` method.
- Use the `'gpt-4o-mini-2024-07-18'` model to get the completions.
- Provide the model as `'model'` variable in the `Client.chat.completions.create` method.
- Provide the text to specify as prompt as a list of dictionaries with the keys `'role'` and `'content'` in the `Client.chat.completions.create` method.
    - In this example we only provide a single message with the user prompt.
    - The `'role'` key should contain `'user'` for user prompt.
    - The `'content'` key should contain the actual prompt as a string.
- The answer/chat-completion is returned as a `ChatCompletion` object.
    - The answer can be accessed using the `ChatCompletion.choices[0].message` attribute, which contains the answer as a string.


In [ ]:
# Setup data for the request
role = "user"
content = "What does OECD stand for?"
messages = [{"role": role, "content": content}]
model = "gpt-4o-mini-2024-07-18"

# Request completion from openai_api
completion = ai_client.chat.completions.create(
    model=model,
    messages=messages
)

# Extract the answer from the completion
answer = completion.choices[0].message.content
print(answer)

### Further information – inspecting `ChatCompletion`

- `ChatCompletion.id` → a unique identifier (`str`) for the chat completion.
- `ChatCompletion.choices` → a list of `Choice` objects; each `Choice` contains:
    - `Choice.finish_reason` → the reason (`str`) why the completion stopped (e.g., 'stop').
    - `Choice.index` → the index (`int`) of the choice in the list.
    - `Choice.message` → the actual message (`ChatCompletionMessage`) returned by the assistant.
        - `ChatCompletionMessage.content` → the content (`str`) of the assistant's response.
        - `ChatCompletionMessage.role` → the role (`str`) of the message sender (e.g., 'assistant').
- `ChatCompletion.created` → the timestamp (`int`) when the completion was created.
- `ChatCompletion.model` → the name (`str`) of the model used for the completion.
- `ChatCompletion.object` → the type (`str`) of the returned object (e.g., 'chat.completion').
- `ChatCompletion.service_tier` → the service tier (`str`) used for the request.
- `ChatCompletion.system_fingerprint` → a unique fingerprint (`str`) for the system.
- `ChatCompletion.usage` → a `CompletionUsage` object containing token usage details:
    - `CompletionUsage.completion_tokens` → the number of tokens (`int`) in the completion.
    - `CompletionUsage.prompt_tokens` → the number of tokens (`int`) in the prompt.
    - `CompletionUsage.total_tokens` → the total number of tokens (`int`) used.
    - `CompletionUsage.completion_tokens_details` → a `CompletionTokensDetails` object with detailed token usage for the completion.
    - `CompletionUsage.prompt_tokens_details` → a `PromptTokensDetails` object with detailed token usage for the prompt.

## Utilizing Chat Models - Stream

In this section, we will use the streaming feature of the OpenAI API to get the answer bit by bit while it is generated.  
We will use the OpenAI API again to get a single answer completion for a given text – "What does 'OECD' stand for?".

### Task
- Use the OpenAI API endpoint `completions` to get a single answer completion for the text – "What does 'OECD' stand for?" using the streaming feature.

### Tips
- Use the `Client.chat.completions.create` method as before.
- Use the `'gpt-4o-mini-2024-07-18'` model to get the completions.
- Set the `stream` parameter to `True` in the `Client.chat.completions.create` method.
- To print the answer bit by bit, use a for-loop to iterate over the `Stream` object (e.g., `for chunk in stream`) returned by the `Client.chat.completions.create` method.
    - The answer can be accessed via `chunk.choices[0]`, which contains the current state of the answer.
    - Use `chunk.choices[0].message.delta` to get only the new part of the answer.
    - Use `chunk.choices[0].message.delta.content` to get the new part of the answer as a string.
- In order to print inline, provide the `end=''` parameter to the print function.

In [ ]:
# Setup data for the request
role = "user"
content = "What does OECD stand for?"
messages = [{"role": role, "content": content}]
model = "gpt-4o-mini-2024-07-18"

# Request completion from openai_api
stream = ai_client.chat.completions.create(
    model=model,
    messages=messages,
    stream=True
)

# Stream the response
for chunk in stream:
    if not chunk.choices:
        continue

    print(chunk.choices[0].delta.content, end="")
print()

## Utilizing Chat Models - Multi Turn

In this section, we will use the OpenAI API to get multiple completions for a given text — conversation-like.

### Task
- Use the OpenAI API endpoint `completions` to get multiple completions.
- In a first step, we will provide the prompt "What does 'OECD' stand for?", and in a second step, we will provide the prompt "What is the purpose of it?".

### Tips
- Use the `Client.chat.completions.create` method as before.
- Use a `history` variable to store the previous messages. Messages from both the user and the assistant are stored in this variable.
- Use the `'gpt-4o-mini-2024-07-18'` model to get the completions.

In [ ]:
# Setup data for the requests
model = "gpt-4o-mini-2024-07-18"
history = []

# Prepare the first message
message_1 = {"role": "user", "content": "What does 'OECD' stand for?"}
history.append(message_1) # Append first message to history
print(f'{history[-1]["role"]} :: \n{history[-1]["content"]}\n---\n\n')
      
# Request first completion from openai_api
completion = ai_client.chat.completions.create(
    model=model,
    messages=history
)

# Extract the answer from the completion
answer_1 = completion.choices[0].message
history.append(answer_1.to_dict())
print(f'{history[-1]["role"]} :: \n{history[-1]["content"]}\n---\n\n')

# Request second completion from openai_api
message_2 = {"role": "user", "content": "What is the purpose of it?"}
history.append(message_2) # Append second message to history
print(f'{history[-1]["role"]} :: \n{history[-1]["content"]}\n---\n\n')

# Prepare the second message
completion = ai_client.chat.completions.create(
    model=model,
    messages=history
)

# Extract the answer from the completion
answer_2 = completion.choices[0].message
history.append(answer_2.to_dict())
print(f'{history[-1]["role"]} :: \n{history[-1]["content"]}\n---\n\n')

## Utilizing Chat Models - System Prompt

In this section, we will use the OpenAI API to get multiple completions for a given text as before, but this time we will provide a system prompt to give some context about the OECD and ensure that the model answers only questions that are within the context of the OECD.

### Task
- Use the OpenAI API endpoint `completions` to get multiple completions as before.
- Use a system prompt to provide some context about the OECD and make sure that the model only answers questions in the context of the OECD.
- Try what happens if you ask a question that is not in the context of the OECD with the system prompt.

### Tips
- Use the `Client.chat.completions.create` method as before.
- As the first message in the `messages` list, provide **always** the system prompt; the `messages` list is passed as input to the `Client.chat.completions.create` method.
    - The `'role'` key should contain `'system'` for the system prompt.
    - The `'content'` key should contain the actual system prompt as a string.
- As the second message in the `messages` list, provide the user prompt.
    - The `'role'` key should contain `'user'` for the user prompt.
    - The `'content'` key should contain the actual user prompt as a string.

In [ ]:
# Setup data for the requests
model = "gpt-4o-mini-2024-07-18"
history = []
system_prompt = {'role' : 'system',
                 'content' : """You are a helpful, friendly and professional assistant helping with questions about the OECD.
                             You can only answer question about the OECD or its context. If a question is outside of this scope, please let the user know that you can't help with that.
                             You can also provide information about the OECD's purpose, history, members, etc. If you don't know the answer, you can say that you don't know.
                             Here are some important facts about the OECD:
                                - The OECD stands for the Organization for Economic Co-operation and Development.
                                - The OECD was founded in 1961 to stimulate economic progress and world trade.
                                - The OECD has 38 member countries, including the United States, the United Kingdom, and Japan.
                                - The OECD is headquartered in Paris, France.
                                - The OECD publishes reports and statistics on a wide range of economic and social issues.
                 """}

history.append(system_prompt)

# Prepare the first message
message_1 = {"role": "user", "content": "What does 'OECD' stand for?"}
history.append(message_1) # Append first message to history
print(f'{history[-1]["role"]} :: \n{history[-1]["content"]}\n---\n\n')
      
# Request first completion from openai_api
completion = ai_client.chat.completions.create(
    model=model,
    messages=history
)

# Extract the answer from the completion
answer_1 = completion.choices[0].message
history.append(answer_1.to_dict())
print(f'{history[-1]["role"]} :: \n{history[-1]["content"]}\n---\n\n')

# Request second completion from openai_api out of scope question
message_2 = {"role": "user", "content": "What is the capital of France?"}
history.append(message_2) # Append second message to history
print(f'{history[-1]["role"]} :: \n{history[-1]["content"]}\n---\n\n')

# Prepare the second message
completion = ai_client.chat.completions.create(
    model=model,
    messages=history
)

# Extract the answer from the completion
answer_2 = completion.choices[0].message
history.append(answer_2.to_dict())
print(f'{history[-1]["role"]} :: \n{history[-1]["content"]}\n---\n\n')

## Build a Simple Chatbot

In this section, we will build a simple chatbot that can answer questions about the OECD. The chatbot uses the OpenAI API and incorporates a **system prompt** to define its behavior and scope. It will **stream responses** in real time and maintain context across multiple turns in the conversation.

### Task
- Use the OpenAI API to build a chatbot that responds to user questions.
- Define a system prompt that restricts the chatbot to only answer questions related to the OECD.
- Accept continuous user input and maintain a history of the conversation.
- Use streaming to display the response in real time.
- End the chat by typing `"break"`.

### Tips
- **Set a system prompt** as the first message in the `history` list. This defines the chatbot’s personality, scope (OECD-related), and behavior when asked questions outside its domain.
- **Initialize an empty `history` list** and append user and assistant messages as the conversation continues.
- Use `input()` to capture user input. Check if the input is `"break"` to end the session and clear the history.
- Call `Client.chat.completions.create()` with:
  - `model`: e.g., `"gpt-4o-mini-2024-07-18"`
  - `messages`: the `history` list containing the conversation so far
  - `stream=True`: to enable real-time response streaming
- Store the full assistant response in a dictionary like `{'role': 'assistant', 'content': ...}` and append it to the `history` list after the message is fully streamed.
- **Maintaining context**: The full `history` list is sent with each new request, enabling the assistant to answer based on previous turns.
- **Exit and reset**: When the user types `"break"`, print a goodbye message and clear the `history` list to reset the session.

In [ ]:
# Setup chatbot data
model = "gpt-4o-mini-2024-07-18"
history = []
system_prompt = {'role' : 'system',
                 'content' : """You are a helpful, friendly and professional assistant helping with questions about the OECD.
                             You can only answer question about the OECD or its context. If a question is outside of this scope, please let the user know that you can't help with that.
                             You can also provide information about the OECD's purpose, history, members, etc. If you don't know the answer, you can say that you don't know.
                             Here are some important facts about the OECD:
                                - The OECD stands for the Organization for Economic Co-operation and Development.
                                - The OECD was founded in 1961 to stimulate economic progress and world trade.
                                - The OECD has 38 member countries, including the United States, the United Kingdom, and Japan.
                                - The OECD is headquartered in Paris, France.
                                - The OECD publishes reports and statistics on a wide range of economic and social issues.
                 """}

history.append(system_prompt)

# Initialize the chatbot
while True:

    # Wait for user input
    user_input = input("You :: \n")
    print("\n---\n")

    # Check if the user wants to break the chat
    if(user_input.lower() == "break"):
        print("Stopped chat and cleaned history.")
        history = []
        break

    # Prepare the user message
    message = {"role": "user", "content": user_input}
    history.append(message) # Append user message to history

    # Request completion from openai_api
    completion = ai_client.chat.completions.create(
        model=model,
        messages=history,
        stream=True
    )

    # Stream the response and extract the answer from the completion
    print("Chatbot :: \n", flush=True)
    answer = {'role' : 'assistant', 'content' : ""}
    
    for chunk in completion:
        if not chunk.choices:
            continue
                   
        if chunk.choices[0].delta.content:
            answer["content"] += chunk.choices[0].delta.content
            print(chunk.choices[0].delta.content, end="")
            
    print("\n---\n")

    # Add the answwer to the history
    history.append(answer) # Append chatbot answer to history#
